### Installing required libraries

In [1]:
!pip install transformers diffusers torch pillow gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Importing required libraries

In [6]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, LMSDiscreteScheduler, \
    PNDMScheduler, DPMSolverMultistepScheduler, DDIMScheduler
import torch
import gradio as gr
from PIL import Image

### Load model pipeline

In [7]:
model_id = "stabilityai/stable-diffusion-2-1"
sd_pipeline = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16
)
sd_pipeline.to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "stabilityai/stable-diffusion-2-1",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

### Defining multiple scheduler algorithms

In [8]:
schedulers = {
    "Euler Discrete": EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "LMS Discrete": LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "PNDM": PNDMScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "DPM Solver Multistep": DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "DDIM": DDIMScheduler.from_pretrained(model_id, subfolder="scheduler"),
}

### Image generating function

In [18]:
def generate_image(
    prompt,
    negative_prompt="",
    num_inference_steps=50,
    guidance_scale=7.5,
    height=512,
    width=512,
    batch_size=1,
    seed=None,
    scheduler="Euler Discrete",
):
    # Set the selected scheduler
    sd_pipeline.scheduler = schedulers.get(scheduler, sd_pipeline.scheduler)

    # Set seed for reproducibility
    generator = torch.manual_seed(seed) if seed else None

    # Generate images
    images = sd_pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        height=height,
        width=width,
        num_images_per_prompt=batch_size,
        generator=generator
    ).images

    return images  # Always return a list, even if batch_size = 1

### Gradio interface

In [19]:
interface = gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Prompt", info="Describe the image you want to generate."),
        gr.Textbox(label="Negative Prompt", info="Specify what to avoid in the image."),
        gr.Slider(step=1, minimum=1, maximum=100, value=50, label="Num Inference Steps",
                  info="Controls the number of denoising steps."),
        gr.Slider(step=0.1, minimum=1, maximum=20, value=7.5, label="Guidance Scale",
                  info="Higher values guide the image closer to the text prompt."),
        gr.Slider(step=64, minimum=256, maximum=1024, value=512, label="Height",
                  info="Set the height of the generated image."),
        gr.Slider(step=64, minimum=256, maximum=1024, value=512, label="Width",
                  info="Set the width of the generated image."),
        gr.Slider(step=1, minimum=1, maximum=4, value=1, label="Batch Size",
                  info="Generate multiple images at once (1-4)."),
        gr.Number(label="Seed", info="Set a seed for reproducibility (Optional)."),
        gr.Dropdown(
            choices=["Euler Discrete", "LMS Discrete", "PNDM", "DPM Solver Multistep", "DDIM"],
            value="Euler Discrete",
            label="Scheduler Algorithm",
            info="Choose the denoising algorithm used for image generation."
        ),
    ],
    outputs=gr.Gallery(label="Generated Images"),
    title="Gen AI Text-to-Image Generator using Stable Diffusion",
    description="Generate AI images from text with customizable settings(parameters).",
)

# Launch Gradio App
interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://43191ee283ba97295e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://43191ee283ba97295e.gradio.live
